In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

## load and preprocessing

In [5]:
img_shape = (800, 600)
ori_img = cv2.imread('image1.jpg', cv2.IMREAD_GRAYSCALE)
ori_img = cv2.resize(ori_img, img_shape)
watermask = cv2.imread('watermask1.jpg', cv2.IMREAD_GRAYSCALE)
watermask = cv2.resize(watermask, img_shape)
result_img = np.zeros(img_shape[::-1], dtype=np.uint8)
print(np.shape(ori_img), np.shape(watermask))

(600, 800) (600, 800)


## display origin image and watermask

In [6]:
def show_image(image):
    cv2.imshow('My Image', image)
    # 按下任意鍵則關閉所有視窗
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [7]:
imgs = np.hstack([ori_img, watermask])
show_image(imgs)

## embed watermask in origin image

In [8]:
for i in range(img_shape[1]):
    for j in range(img_shape[0]):
        ori_b_pixel = "{:08b}".format(ori_img[i][j])
        watermask_b_pixel = "{:08b}".format(watermask[i][j])
        embed_b_img = ori_b_pixel[:4] + watermask_b_pixel[:4]
        # print(result_b_img, ori_b_pixel, watermask_b_pixel)
        result_img[i][j] = int(embed_b_img,2)

## display result

In [54]:
origin_result_image = np.hstack([ori_img, result_img])
show_image(origin_result_image)

## save mixing image

In [39]:
cv2.imwrite('mixing_img.jpg', result_img)

True

## extract image

In [55]:
extract_ori_img = np.zeros(img_shape[::-1], dtype=np.uint8)
extract_ori_watermask = np.zeros(img_shape[::-1], dtype=np.uint8)

In [57]:
mixing_img = result_img
# mixing_img = cv2.imread('mixing_img.jpg', cv2.IMREAD_GRAYSCALE)
show_image(mixing_img)

In [58]:
for i in range(img_shape[1]):
    for j in range(img_shape[0]):
        mixing_b_pixel = "{:08b}".format(mixing_img[i][j])
        extract_b_ori_img = mixing_b_pixel[:4] + '0000'
        extract_b_ori_watermask = mixing_b_pixel[-4:] +'0000'
        extract_ori_img[i][j] = int(extract_b_ori_img,2)
        extract_ori_watermask[i][j] = int(extract_b_ori_watermask,2)

In [60]:
imgs = np.hstack([extract_ori_img, extract_ori_watermask])
show_image(imgs)

In [16]:
print(type('0000'))

<class 'str'>


In [19]:
a = "{:08b}".format(2)
print(type(a))

<class 'str'>


# Complete Source Code

In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

image_shape = (800, 600)

class watermask_module:
    def __init__(self):
        self.ori_backgd = np.array([])
        self.ori_watermask = np.array([])
        self.__embed_image = np.array([])
        self.__extract_backgd = np.array([])
        self.__extract_watermask = np.array([])

    def embed_wm_in_img(self, background, watermask, emd_bits, img_shape = image_shape):
        background = cv2.resize(background, img_shape)
        watermask = cv2.resize(watermask, img_shape)
        self.ori_backgd = background
        self.ori_watermask = watermask
        self.embed_image = np.zeros(img_shape[::-1], dtype=np.uint8)

        for i in range(img_shape[1]):
            for j in range(img_shape[0]):
                background_b_pixel = "{:08b}".format(background[i][j])
                watermask_b_pixel = "{:08b}".format(watermask[i][j])
                embed_b_img = background_b_pixel[:(8-emd_bits)] + watermask_b_pixel[:emd_bits]
                self.embed_image[i][j] = int(embed_b_img,2)
        return self.embed_image

    def extract_wm_in_img(self, embeded_img, emd_bits):
        self.__extract_backgd = np.zeros(img_shape[::-1], dtype=np.uint8)
        self.__extract_watermask = np.zeros(img_shape[::-1], dtype=np.uint8)
        
        for i in range(img_shape[1]):
            for j in range(img_shape[0]):
                embeded_b_pixel = "{:08b}".format(embeded_img[i][j])
                extract_background_b_pixel = "{:0<8}".format(embeded_b_pixel[:(8-emd_bits)])
                extract_watermask_b_pixel = "{:0<8}".format(embeded_b_pixel[-emd_bits:])
                self.__extract_backgd[i][j] = int(extract_background_b_pixel,2)
                self.__extract_watermask[i][j] = int(extract_watermask_b_pixel,2)
        return self.__extract_backgd, self.__extract_watermask



In [19]:
def show_image(desc, image):
    cv2.imshow(str(desc), image)
    # 按下任意鍵則關閉所有視窗
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [3]:
img_shape = (800, 600)
ori_img = cv2.imread('image1.jpg', cv2.IMREAD_GRAYSCALE)
watermask = cv2.imread('watermask1.jpg', cv2.IMREAD_GRAYSCALE)

In [23]:
# one bit embeding
one_bit_embed = watermask_module()
embed_1b_image = one_bit_embed.embed_wm_in_img(ori_img, watermask, 1)
ext_1b_ori_backbg, ext_1b_ori_watermask = one_bit_embed.extract_wm_in_img(embed_1b_image, 1)

imgs = np.hstack([one_bit_embed.ori_backgd, embed_1b_image])
show_image('1b origin(right) and embeded(left) background', imgs)
imgs = np.hstack([one_bit_embed.ori_backgd, ext_1b_ori_backbg])
show_image('1b origin(right) and extract(left) background', imgs)
imgs = np.hstack([one_bit_embed.ori_watermask, ext_1b_ori_watermask])
show_image('1b origin(right) and extract(left) watermask', imgs)

In [24]:
# two bit embeding
two_bit_embed = watermask_module()
embed_2b_image = two_bit_embed.embed_wm_in_img(ori_img, watermask, 2)
ext_2b_ori_backbg, ext_2b_ori_watermask = two_bit_embed.extract_wm_in_img(embed_2b_image, 2)

imgs = np.hstack([two_bit_embed.ori_backgd, embed_2b_image])
show_image('2b origin(right) and embeded(left) background', imgs)
imgs = np.hstack([two_bit_embed.ori_backgd, ext_2b_ori_backbg])
show_image('2b origin(right) and extract(left) background', imgs)
imgs = np.hstack([two_bit_embed.ori_watermask, ext_2b_ori_watermask])
show_image('2b origin(right) and extract(left) watermask', imgs)


In [15]:
# three bit embeding
three_bit_embed = watermask_module()
embed_3b_image = three_bit_embed.embed_wm_in_img(ori_img, watermask, 3)
ext_3b_ori_backbg, ext_3b_ori_watermask = three_bit_embed.extract_wm_in_img(embed_3b_image, 3)

imgs = np.hstack([three_bit_embed.ori_backgd, embed_3b_image])
show_image('3b origin(right) and embeded(left) background', imgs)
imgs = np.hstack([three_bit_embed.ori_backgd, ext_3b_ori_backbg])
show_image('3b origin(right) and extract(left) background', imgs)
imgs = np.hstack([three_bit_embed.ori_watermask, ext_3b_ori_watermask])
show_image('3b origin(right) and extract(left) watermask', imgs)


In [14]:
show_image(ext_ori_image)
show_image(ext_ori_watermask)

In [31]:
show_image(result)

In [40]:
print(type("{:0<8b}".format(3)))


<class 'str'>
